## GNN Explainer

Here we visualize the explainer results and analyze specific graph theoretic properties

In [1]:
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
logdir = '../log/'
# Here all the required .npy files are directly under log
expdir = ''

In [3]:
# Load produced masks
dirs = os.listdir(os.path.join(logdir, expdir))
dirs

['enron_base_h20_o20',
 'grad',
 'graph',
 'mask',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_400graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_405graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_410graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_415graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_420graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_425graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_430graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_435graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_440graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_445graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_450graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_455graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_460graph_idx_-1.npy',
 'masked_adj_syn2_base_h20_o20_explainnode_idx_465gr

In [4]:
masks = []

# This would print all the files and directories
for file in dirs:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy':
        # print(file)
        masks.append(file)
type(masks[0])

str

In [5]:
def show_adjacency_full(mask, ax=None):
    """Plot full adjacency matrix of the mask

    Args:
        mask (str): Filename containing the adjacency matrix of the mask
        ax (Axes class object, optional): Axis. Defaults to None.

    Returns:
        numpy matrix: Full adjacency matrix of the mask
    """
    # Obtain adjacency matrix from the filename sent in var "mask"
    adj = np.load(os.path.join(logdir, expdir, mask), allow_pickle=True)
    # if ax is None:
    #     plt.figure()
    #     plt.imshow(adj);
    # else:
    #     ax.imshow(adj)
    return adj

In [6]:
def filter_adj(adj,threshold=0.8):
    """Filter out the values in adjacency matrix that are greater than threshold. Fix the others as 0

    Args:
        adj (numpy matrix): Adjacency matrix of mask that we have to filter
        threshold (float): Filter value

    Returns:
        numpy matrix: Filtered adjacency matrix
    """
    filt_adj = adj.copy()
    filt_adj[adj<threshold] = 0
    return filt_adj

In [ ]:
# for m in masks: 
#     fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
#     plt.title(str(m))
    
#     # Full adjacency
#     ax1.set_title('Full Adjacency mask')
#     adj = show_adjacency_full(m, ax=ax1)
    
#     # Filtered adjacency
#     f = filter_adj(adj)
#     ax2.set_title('Filtered Adjacency mask')
#     ax2.imshow(f)
    
#     # Plot subgraph
#     ax3.set_title("Subgraph")
#     G = nx.from_numpy_array(f)
#     G.remove_nodes_from(list(nx.isolates(G)))
#     nx.draw(G, ax=ax3)
# print(type(ax1))

### Analyzing Graph theoretic properties of each mask

Here we measure the following quantities:
- Nodes: |V|
- Edges: |E|
- Avg degree
- Diameter of the graph
- Sparsity measure: edge density = |E|/C(|V|,2)
- Node centrality measure: Betweenness centrality

In [24]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def normalize(A):
    scale_factor = A.max() - A.min()
    B = np.ones_like(A)*A.min()

    A = (A - B)/scale_factor
    return A

def centrality(adj):
    # Get the unweighted adjacency matrix
    adj1 = adj.copy()
    adj1[adj>0] = 1

    # Get the unweighted graph
    unweighted_G = nx.from_numpy_matrix(np.matrix(adj1))

    # For now we use edge-path centrality later check
    # Information centrality metric
    ebc = nx.edge_betweenness_centrality(unweighted_G)

    # Create a symmetric matrix A with A[i][j] is the betweeness score of edge (i,j)
    A = np.zeros_like(adj1)
    v = unweighted_G.number_of_nodes()
    for i in range(v):
        for j in range(i):
            if (j,i) in ebc.keys():
                # The keys in ebc are (a,b) with a < b
                A[i][j] = A[j][i] = ebc[(j,i)]
    
    # Now that we have got A, we see if the importance given by GNNExplainer
    # has some relation with the centrality scores in A
    A1 = normalize(A)
    B1 = normalize(adj)
    # For coherence check the following two links:
    # 1) https://www.geeksforgeeks.org/cosine-similarity-calculation-between-two-matrices-in-matlab/
    # 2) https://math.stackexchange.com/questions/507742/distance-similarity-between-two-matrices#:~:text=If%20we%20have%20two%20matrices,squares%20of%20all%20singular%20values.
    coherence = 1/(np.linalg.norm(A1 - B1,2))**2

    return coherence

def graph_prop(adj):
    # Edge weighted graph
    weighted_G = nx.from_numpy_matrix(np.matrix(adj))

    v = weighted_G.number_of_nodes()
    e = weighted_G.number_of_edges()
    avg_degree = float('%.3f'%(2*e/v))
    diameter = nx.diameter(weighted_G)
    sparsity = float('%.3f'%(2*e/(v*(v-1))))
    coherence = float('%.3f'%(centrality(adj)))

    return [v,e,avg_degree,diameter,sparsity,coherence]

In [25]:
data = []

for m in masks:
    # Full adjacency matrix
    adj = show_adjacency_full(m)
    # Get the metrics of each mask
    data.append(graph_prop(adj))

In [31]:
import pandas as pd

# Convert the above data into a data frame
df = pd.DataFrame(data, columns=["Order","Size","Average Degree","Diameter","Sparsity","Coherence"])


df.describe()

,Order,Size,Average Degree,Diameter,Sparsity,Coherence
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,153.483333,479.216667,6.062067,5.916667,0.042633,0.412300
std,52.644472,219.189268,0.841364,0.278718,0.009872,0.121684
min,84.000000,217.000000,4.673000,5.000000,0.023000,0.205000
25%,113.000000,314.750000,5.407250,6.000000,0.035750,0.302250
50%,147.500000,446.000000,5.998000,6.000000,0.042000,0.421000
75%,165.250000,571.000000,6.813000,6.000000,0.050250,0.510250
max,337.000000,1321.000000,7.840000,6.000000,0.064000,0.649000
